# Set Up functions for getting Data

In [1]:
import os
os.chdir('/home/tzd/fs03/d1/tzd/Data_analysis/RAworkflow')
%run ./My_Modules/DataSetUP10VAR.ipynb
#FmodelDF


boxes().shape[0]
# getNoASGM():       => Hg0no_ASGM dataframe; in correct units
# get_model(region): => Hg0_modified; in correct units (takes gridbox region in Peru as input)
# getLoRes():        => Hg0 baseline with ASGM low resulution in correct units
# getHiRes():        => Hg0 baseline with ASGM high resulution in correct units
# getNoASGM():       => Hg0 baseline without ASGM low resulution in correct units
# get_obs_d():       => obs_Hg0; original observations daily mean values
# get_detrended_obs(): => Obs_detrended; return detrended observations
# get_detrended_model(model): => This function takes in a dataframe of model outputs 
                                and returns the detrended version of that model output
# getObsIQR(obs): This function takes observations and returns the IQR 
# getIQR_Dd(model): This function takes in a dataframe of model outputs and returns the detrended IQR
# getIQR(model): => This function takes in a dataframe of model outputs and returns the IQR
#

/home/tzd/anaconda3/envs/Jlab/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/home/tzd/anaconda3/envs/Jlab/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/home/tzd/anaconda3/envs/Jlab/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future ve

# Import modules 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
from scipy.stats import norm,iqr,bootstrap


# Formular for Calculating Emission Signal from each gridbox


$$\small{Y_{signal}} =\small\frac{(X_{modified}-X_{base})}{(M_1-M_0)}\small{(M-M_o)}$$

* $\small{Y_{signal}}$ : The Emission signal from a gridbox

* $X_{modified}$ : The modified emissions GEOS Chem output __(model output)__
* $X_{base}$ : The baseline emissions GEOS Chem output __(model output)__
* $M_1$ : The Hg mass after modification __(known)__
* $M_0$ : The Hg mass before modification __(known)__
* $M$ : The intended Hg emissions mass __(unknown)__

The above equation can be expanded in to separate the uknowns from the known information so that $M$ can be used as a parameter in our optimization

$$\small{Y_{signal}} =\small\frac{(X_{modified}-X_{base})}{(M_1-M_0)}M-\small\frac{(X_{modified}-X_{base})}{(M_1-M_0)}M_o$$

let $$W_{region}=\small\frac{(X_{modified}-X_{base})}{(M_1-M_0)}$$

hence the above equation becomes:
$$\small{Y_{signal}} =W_{region}M-W_{region}M_o$$

## Optimization of combined signal from differnt gridboxes 


The modified model output at Chalcataya accounting for signals from the different regions is given by the equation: 

$$Hg_{(modified)}= HgSignal_{(MdD )}+ HgSignal_{(S-Puno)} + HgSignal_{(N-Puno)}+ HgSignal_{(Apurimac)}+ HgSignal_{(Arequipa)}+HgBaseline_{(No-ASGM)}$$

$$Hg_{(modified)}= (W_{(MdD)}M-W_{(MdD)}M_o)+ (W_{(S-Puno)}M-W_{(S-Puno)}M_o) + (W_{(N-Puno)}M-W_{(N-Puno)}M_o)+ (W_{(Apurimac)}M-W_{(Apurimac)}M_o)+  (W_{(Arequipa)}M-W_{(Arequipa)}M_o)+HgBaseline_{(No-ASGM)}$$

$M$  is the parameter that we want to modify hence let $θ=M$ and combine all the  $−𝑊_{(𝑀𝑑𝐷)}𝑀$  in to one constant  $Z$  which leads to the simplified equation:

$$Hg_{(modified)}= (HgBaseline_{(no-ASGM)}θ_0 + Z\theta_1 + W_{(mdd)}θ_2+ W_{(s-pun)}θ_3 + W_{(n-pun)}θ_4+ W_{(aprc)}θ_5+  W_{(aqpa)}θ_6$$



# Model Function Equations

## Representation

$$Y = f(theta)$$

$$let\ Constant\ = (HgBaseline_{(no-ASGM)}θ_0 + Z\theta_1$$ 
$$where\ \theta_0 =\theta_1=1$$

$$Y=\begin{bmatrix} Constant & W_{(mdd)} & W_{(s-pun)} & W_{(n-pun)} &W_{(aprc)} &W_{(aqpa)}\end{bmatrix} × \begin{bmatrix} \theta_0 \\θ_1 \\ θ_2\\ θ_3\\ θ_4\\ θ_5 \end{bmatrix}$$ 


# Analysis

## Initialize the constants

In [3]:
def initEmcee(RefSiteNum):
    #RefSiteNum = RefSiteNum
    regions = ['spun','npun','mdd','aqp','apr']
    ConstantTerm=getConst(RefSiteNum,regions)
    Spun_sigs=multiSiteSignal('spun',RefSiteNum).to_numpy()
    Npun_sigs = multiSiteSignal('npun',RefSiteNum).to_numpy()
    Mdd_sigs =multiSiteSignal('mdd',RefSiteNum).to_numpy()
    Aqp_sigs = multiSiteSignal('aqp',RefSiteNum).to_numpy()
    Apr_sigs =multiSiteSignal('apr',RefSiteNum).to_numpy()
    return ConstantTerm,Spun_sigs,Npun_sigs,Mdd_sigs,Aqp_sigs,Apr_sigs

## Set Up the Model

In [4]:
def set_metric(metric_type):
    if metric_type=='95th':
        print("model metric is 95th percentile")
        def model(theta):
            """Dummy model used here for Bayesian analysis (replace with actual GEOS-Chem function):
                Model: Y = f(theta) where theta is a set of parameters:
                    Emissions: E1, E2, and E3
            """
            Espun, Enpun, Emdd, Eaqp, Eapr=theta
            
            modified=ConstantTerm+ (Espun*Spun_sigs) + (Enpun*Npun_sigs) + (Emdd*Mdd_sigs) +(Eaqp*Aqp_sigs) +Eapr*Apr_sigs 
            
            summary =pd.DataFrame(modified)
            model_out = list(summary.apply(find_95th))

            return model_out
    elif metric_type=='mean':
        print("model metric is mean")
        def model(theta):
            Espun, Enpun, Emdd, Eaqp, Eapr=theta
            
            modified=ConstantTerm+ (Espun*Spun_sigs) + (Enpun*Npun_sigs) + (Emdd*Mdd_sigs) +(Eaqp*Aqp_sigs) +Eapr*Apr_sigs 
            model_out = list(modified.mean(axis=0))

            return model_out
    else:
        print("model metric is IQR")
        def model(theta):
            Espun, Enpun, Emdd, Eaqp, Eapr=theta
            
            modified=ConstantTerm+ (Espun*Spun_sigs) + (Enpun*Npun_sigs) + (Emdd*Mdd_sigs) +(Eaqp*Aqp_sigs) +Eapr*Apr_sigs 
            model_out = list(iqr(modified, axis=0))

            return model_out
    return model

### Sanity Check

#### Initialize constants 

In [5]:
# ConstantTerm,Spun_sigs,Npun_sigs,Mdd_sigs,Aqp_sigs,Apr_sigs=initEmcee(2)

#### Run model function

In [6]:
# #Testing Modelling function
# thet = [10,13,60,25,1]
# model = set_metric('90th')

# print('The 90th for [10,13,60,25,1] percentiles are')
# print(model(thet))
# print('The 90th for [14,13,40,25,10] percentiles are')
# print(model([14,13,40,25,10]))

# print('\n The means are')
# model = set_metric('mean')
# print(model(thet))
# print('\n The means for [14,13,40,25,10] are:')
# print(model([14,13,40,25,10]))

# print('\n The IQRs are')
# model = set_metric('iqr')
# print(model(thet))
# print('\n The IQRs for [14,13,40,25,10] are:')
# print(model([14,13,40,25,10]))



## Set Up MCMC Parameters

In [7]:
def lnlike(theta, y_obs, y_err):

   """Calculating log likelihood assuming iid Gaussian errors
       Parameters
       ----------
       theta : parameters
       y_obs : observed value (e.g. of IQR)
       y_err : error in observed value (e.g. of IQR)"""
   y = model(theta) # calculate modelled output
    # calculate log-likelihood that observed value is drawn from normal
    # distribution with mean y (modelled value) and sigma = y_err
   # if len(y)== 1:
   #    LnLike = norm.logpdf(y_obs, y, y_err)
   # else:
   #    print('somehting is wrong')
   LnLike = sum(norm.logpdf(y_obs, y, y_err))
   return LnLike

In [8]:
def lnprior(theta):
    """Apply prior assumption bounds (i.e. upper and lower bounds of variables)
       Parameters
       ----------
       theta : parameters    
    """
    
    E1, E2, E3, E4, E5 = theta
    
    # Prior assumptions for emissions - vary between 0 and 100 (can adjust this):
    min_emiss = 0
    max_emiss = 100 
    
    if E1 <= min_emiss or E2 <= min_emiss or E3 <= min_emiss or E4 <= min_emiss or E5 <= min_emiss: # enforce lower bound
        return -np.inf
    elif E1 >= max_emiss or E2 >= max_emiss or E3 >= max_emiss or E4 >= max_emiss or E5 >= max_emiss: # enforce upper bound
        return -np.inf
    else: # all values within prior bounds
        return 0.0
    

In [9]:
def lnprob(theta, y_obs, y_err):
    """Overall function that calculates log likelihood probability
       Parameters
       ----------
       theta : parameters  
       y_obs : observed value (e.g. of IQR)   
       y_err : error in observed value (e.g. of IQR)      
    """
    
    lp = lnprior(theta) #call lnprior
    if not np.isfinite(lp): # check if lp is non-zero:
        return -np.inf
    else: #recall if lp not -inf, its 0, so this just returns likelihood
        return lp + lnlike(theta, y_obs, y_err) 


### Sanity Check

In [10]:
# import numpy as np
# rng = np.random.default_rng()
# from scipy.stats import norm
# dist = norm(loc=2, scale=4)  # our "unknown" distribution
# data = dist.rvs(size=100, random_state=rng)


In [11]:
# import numpy as np
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from sklearn.linear_model import LinearRegression
# X=np.array(list(obs_Hg0_d['value'])).reshape(-1, 1)
# lr = LinearRegression()
# imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman',random_state=0)
# X=imp.fit_transform(X)
# X

In [12]:
# get_detrended_obs()[get_detrended_obs()['value'].isnull()==True]

In [13]:
# def fun_95th(sample,axis): #function for calculating iqr
#     iqr = np.quantile(sample, 0.975,axis) - np.quantile(sample, 0.025,axis) 
#     return iqr

In [14]:
# refsitenum = 0
# data = get_detrended_obs().to_numpy()
# res = bootstrap((data,), fun_95th)
# np.array(res.standard_error).mean()

In [15]:
# refsitenum = 1
# data=getBoxesAround(refsitenum).values
# res = bootstrap((data,), np.mean)
# (np.array(res.standard_error)).mean()

In [ ]:
# refsitenum = 1
# ConstantTerm,Spun_sigs,Npun_sigs,Mdd_sigs,Aqp_sigs,Apr_sigs=initEmcee(refsitenum)
# initial = np.array([7.75, 11.66, 1.39, 13.63, 18.99])
# print(np.array(getBoxesAround(refsitenum).mean()))
# model = set_metric('mean')
# print('model output')
# model(initial)

## Set Up Run

In [16]:
def fun_95th(sample,axis): #function for calculating iqr
    iqr = np.quantile(sample, 0.975,axis) - np.quantile(sample, 0.025,axis) 
    return iqr

In [17]:
# fun_95th(get_detrended_obs().value,0)

In [18]:
def fun_iqr(sample,axis): #function for calculating 95th percentile range
    result = np.quantile(sample, 0.75,axis) - np.quantile(sample, 0.25,axis) 
    return result

In [21]:
def run_simulation(RefSiteNum,metric,init):
    
    
    #theta_true = (E1_true, E2_true, E3_true,E4_true, E5_true)
    if metric == '95th':
        y_true = fun_95th(obs_Hg0_d['value'],0)
        data = obs_Hg0_d['value'].to_numpy()
        res = bootstrap((data,), fun_95th)
        y_err = (np.array(res.standard_error)).mean()
        print(f"The true value of y is: {y_true} \n ")
        print(f"the error is: {y_err}")
    elif metric == 'mean':
        y_true = obs_Hg0_d['value'].mean()
        data = obs_Hg0_d['value'].to_numpy()
        res = bootstrap((data,), np.mean)
        y_err = (np.array(res.standard_error)).mean()
        print(f"The true value of y is: {y_true} \n")
        print(f"the error is: {y_err}")
    else:
        y_true = getObsIQR(obs_Hg0_d)
        data=obs_Hg0_d['value']
        res = bootstrap((data,), fun_iqr)
        y_err = (np.array(res.standard_error)).mean()

    initial = init

    ndim = len(initial)
    # hyperparameter for jump size between guesses
    step_size = 1e-5 
    # number of chains to run simultaneously and their length
    nwalkers = 100
    niter = 1000

    # setting initial guesses for all of the chains
    p0 = [np.array(initial) + step_size * np.random.randn(ndim) for i in range(nwalkers)]

    sampler = emcee.EnsembleSampler(nwalkers,ndim,lnprob, args=(y_true, y_err))
    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 500)
    sampler.reset()

    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter)
    return sampler

In [20]:
def run_simulation_error(RefSiteNum,metric,init,error):
    
    
    #theta_true = (E1_true, E2_true, E3_true,E4_true, E5_true)
    if metric == '95th':
        y_true = fun_95th(get_detrended_obs().value,0)
        y_err = error
        print(f"The true value of y is: {y_true} \n ")
        print(f"the error is: {y_err}")
    elif metric == 'mean':
        y_true = obs_Hg0_d['value'].mean()
        y_err = error
        print(f"The true value of y is: {y_true} \n")
        print(f"the error is: {y_err}")
    else:
        y_true = getObsIQR(get_detrended_obs())
        y_err = error

    initial = init

    ndim = len(initial)
    # hyperparameter for jump size between guesses
    step_size = 1e-5 
    # number of chains to run simultaneously and their length
    nwalkers = 50
    niter = 1000

    # setting initial guesses for all of the chains
    p0 = [np.array(initial) + step_size * np.random.randn(ndim) for i in range(nwalkers)]

    sampler = emcee.EnsembleSampler(nwalkers,ndim,lnprob, args=(y_true, y_err))
    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 500)
    sampler.reset()

    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter)
    return sampler